In [1]:
import pandas as pd
import re
import csv

In [2]:
drug_list = pd.read_csv("eHealth-Drugs.csv", sep="\t")
drug_list

,Code System ID,Code System Version,Concept Code,Description (FSN)
0,2.16.840.1.113883.6.73,2017-01,A,ALIMENTARY TRACT AND METABOLISM
1,2.16.840.1.113883.6.73,2017-01,A01,STOMATOLOGICAL PREPARATIONS
2,2.16.840.1.113883.6.73,2017-01,A01A,STOMATOLOGICAL PREPARATIONS
3,2.16.840.1.113883.6.73,2017-01,A01AA,Caries prophylactic agents
4,2.16.840.1.113883.6.73,2017-01,A01AA01,sodium fluoride
5,2.16.840.1.113883.6.73,2017-01,A01AA02,sodium monofluorophosphate
6,2.16.840.1.113883.6.73,2017-01,A01AA03,olaflur
7,2.16.840.1.113883.6.73,2017-01,A01AA04,stannous fluoride
8,2.16.840.1.113883.6.73,2017-01,A01AA30,combinations
9,2.16.840.1.113883.6.73,2017-01,A01AA51,"sodium fluoride, combinations"


In [3]:
patterns = '(?P<Chemical_Substance>[A-Z][0-9]+[A-Z][A-Z][0-9]+)|(?P<Chemical_Subgroup>[A-Z][0-9]+[A-Z][A-Z])|(?P<Pharmacological_Subgroup>[A-Z][0-9]+[A-Z])|(?P<Therapeutical_Subgroup>[A-Z][0-9]+)|(?P<Anatomical_Main_Group>[A-Z])'

In [4]:
drug_regex = {}

In [5]:
pattern_re = re.compile(patterns)

In [6]:
concept_code = drug_list["Concept Code"].str.extract(patterns, expand=True)
concept_code = concept_code[concept_code.columns[::-1]]

In [27]:
concept_code_boolean = concept_code.notnull()
concept_code_boolean

,Anatomical_Main_Group,Therapeutical_Subgroup,Pharmacological_Subgroup,Chemical_Subgroup,Chemical_Substance
0,True,False,False,False,False
1,False,True,False,False,False
2,False,False,True,False,False
3,False,False,False,True,False
4,False,False,False,False,True
5,False,False,False,False,True
6,False,False,False,False,True
7,False,False,False,False,True
8,False,False,False,False,True
9,False,False,False,False,True


In [62]:
d_list = []
d_name = []
for i in concept_code_boolean:
    d_name.append(i)
    d_list.append(drug_list[concept_code_boolean[i]]["Description (FSN)"])

In [90]:
drug_classified = pd.concat(d_list, axis=1)
drug_classified.columns = d_name
drug_classified = drug_classified.fillna("-")


In [75]:
a = ["-"]*(len(drug_classified.columns)-1)
for i in enumerate(a):
    print(i)

(0, '-')
(1, '-')
(2, '-')
(3, '-')


In [93]:
prev_row_val = ["-"]*(len(drug_classified.columns))
for row in drug_classified.iterrows():
    row_index = row[0]
    row_val = row[1].values
    for i in range(len(row_val)):
        if not row_val[i] == "-" and not row_val[i] == prev_row_val[i]:
            prev_row_val[i] = row_val[i]
            prev_row_val[(i+1):] = ["-"]*len(prev_row_val[(i+1):])
        key = drug_classified.columns[i]
        drug_classified[key][row_index] = prev_row_val[i]



In [94]:
drug_classified

,Anatomical_Main_Group,Therapeutical_Subgroup,Pharmacological_Subgroup,Chemical_Subgroup,Chemical_Substance
0,ALIMENTARY TRACT AND METABOLISM,-,-,-,-
1,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,-,-,-
2,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,-,-
3,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,-
4,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,sodium fluoride
5,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,sodium monofluorophosphate
6,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,olaflur
7,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,stannous fluoride
8,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,combinations
9,ALIMENTARY TRACT AND METABOLISM,STOMATOLOGICAL PREPARATIONS,STOMATOLOGICAL PREPARATIONS,Caries prophylactic agents,"sodium fluoride, combinations"


In [95]:
drug_classified.to_csv("drug_classified1.tsv", sep="\t")

In [102]:
drug_final_list = drug_classified[drug_classified["Chemical_Substance"] != "-"]

In [103]:
drug_final_list.to_csv("drug_classified.tsv", sep="\t")

In [104]:
with open("Anatomical_Therapeutic_Chemical_Classification.gmt", "w") as w:
    prev_sig = ""
    gmt_row = ""
    for row in drug_final_list.iterrows():
        row_val = row[1].values
        sMol = row_val[-1]
        sMol_sig = "_".join(row_val[:-1])
        if not sMol_sig == prev_sig:
            prev_sig = sMol_sig
            if gmt_row.strip():
                w.write(gmt_row+"\n")
            gmt_row = sMol_sig + "\t"
        gmt_row = gmt_row + "\t" + sMol
    else:
        w.write(gmt_row)